In [63]:
import numpy as np
import pandas as pd
import tensorflow as tf

from gensim.models import KeyedVectors
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Concatenate, Conv1D, Dense, Embedding, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from unidecode import unidecode

np.random.seed(42)
tf.compat.v1.random.set_random_seed(42)

# Data Ingestion

In [2]:
def load_data(file_path, language, columns, sample=0):
    df = pd.read_parquet(file_path)
    if sample == 0:
        return df[df["language"] == language][columns]
    else:
        return df[df["language"] == language].groupby("category").apply(
            lambda cat: cat.sample(frac=sample)
        ).reset_index(drop=True)[columns]

In [56]:
train_data = "../data/meli/train_reliable.parquet"
trainu_data = "../data/meli/train_unreliable.parquet"
dev_data = "../data/meli/dev.parquet"
test_data = "../data/meli/test.parquet"
language = "spanish"
word_vectors = f"../data/{language}/{language}-word2vec.bin.gz"
columns = ["words", "pos", "category"]
max_sequence_len = 20
pos_vector_size = 3
filters = [2, 3, 5, 10]
filter_count = 128
network_size = 1
layer_size = 128
activation = "relu"
padding = "valid"

In [4]:
%%time
train_df = pd.concat([
    load_data(train_data, language, columns), 
    load_data(trainu_data, language, columns, 0.05)
])
train_df.head()

CPU times: user 1min 27s, sys: 35.1 s, total: 2min 2s
Wall time: 1min 17s


,words,pos,category
267,"[play, station, 2, +, volante, hooligans, .]","[VERB, NOUN, NUM, PROPN, ADV, ADJ, PUNCT]",GAME_CONSOLES
272,"[pilas, energizer, max, aa, x1, -, tira, x, 20...","[NOUN, VERB, ADP, DET, PROPN, PUNCT, VERB, CON...",CELL_BATTERIES
287,"[afeitadora, electrica, philips, hq6904, +, en...","[ADJ, ADJ, ADP, NOUN, PROPN, VERB, PROPN]",SHAVING_MACHINES
386,"[estufa, calefactor, volcan, 2500, kcal, /, h,...","[NOUN, NOUN, VERB, NOUN, NOUN, PUNCT, CONJ, NU...",HOME_HEATERS
449,"[reloj, pared, vox, tronic, blanco, numeros, 2...","[NOUN, NOUN, NOUN, NOUN, ADJ, NOUN, NUM, ADP, ...",WALL_CLOCKS


In [5]:
%%time
dev_df = pd.concat([
    load_data(dev_data, language, columns)
])
dev_df.head()

CPU times: user 12.6 s, sys: 10.3 s, total: 22.8 s
Wall time: 13.2 s


,words,pos,category
11,"[rosario, contador, de, billetes, uv, /, mg, d...","[ADJ, NOUN, ADP, NOUN, ADJ, PUNCT, ADV, VERB, ...",BILL_COUNTERS
23,"[portón, de, chapa, 3, hojas, mtr, 2.50, sin, ...","[NOUN, ADP, NOUN, NUM, NOUN, PRON, NUM, ADP, N...",GARAGE_DOORS
26,"[base, simil, cemento, -, 30, cm, x, 5, mm]","[NOUN, VERB, NOUN, PUNCT, NUM, NOUN, CONJ, NUM...",CAKE_TOPPERS
34,"[disfraz, de, general, grievous, para, adultos...","[NOUN, ADP, NOUN, ADJ, ADP, NOUN, PUNCT, NOUN,...",COSTUMES
45,"[hermoso, árbol, de, navidad, en, madera, de, ...","[ADJ, NOUN, ADP, NOUN, ADP, NOUN, ADP, NOUN]",CHRISTMAS_TREES


In [9]:
%%time
test_df = pd.concat([
    load_data(test_data, language, ["id", "words", "pos"])
])
test_df.head()

CPU times: user 983 ms, sys: 184 ms, total: 1.17 s
Wall time: 965 ms


,id,words,pos
9,9,"[disco, rigido, externo, western, digital, ele...","[NOUN, ADJ, ADJ, NOUN, ADJ, NOUN, NUM, ADP, PR..."
10,10,"[picadora, de, carne, fineschi, legitima, 32]","[NOUN, ADP, NOUN, NOUN, ADJ, NUM]"
14,14,"[set, barreta, automotor, bremen, x3, unid, .,...","[NUM, NOUN, NOUN, ADJ, PROPN, VERB, PUNCT, NOU..."
15,15,"[miel, organica, x250gr, ., (, sin, tacc, )]","[NOUN, ADJ, PROPN, PUNCT, PUNCT, ADP, NOUN, PU..."
19,19,"[bandeja, giradiscos, omnitronic, bd1320]","[VERB, NOUN, VERB, ADJ]"


In [6]:
%%time
w2v = KeyedVectors.load_word2vec_format(word_vectors, binary=True)

CPU times: user 34.1 s, sys: 828 ms, total: 34.9 s
Wall time: 33.6 s


# Data Preprocessing

## Label Encoding

In [7]:
%%time
def label_encoder(*dfs):
    labels = pd.concat(dfs)["category"].tolist()
    lbl_enc = LabelEncoder().fit(labels)

    return lbl_enc

lbl_enc = label_encoder(train_df, dev_df)

train_df["target"] = lbl_enc.transform(train_df["category"])
train_df.drop(["category"], axis=1, inplace=True)

dev_df["target"] = lbl_enc.transform(dev_df["category"])
dev_df.drop(["category"], axis=1, inplace=True)

CPU times: user 7.51 s, sys: 1.04 s, total: 8.55 s
Wall time: 2.25 s


## Words/PoS to index

In [14]:
%%time
def words_to_idx(all_words, w2v, null_token="<NULL>", unknown_token="<UNK>"):
    word_index = {word for words in all_words for word in words if word in w2v}
    word_index = {word: idx for idx, word in enumerate(sorted(word_index), start=1)}
    word_index[null_token] = 0
    word_index[unknown_token] = len(word_index)

    return word_index

def pos_to_idx(all_pos, null_pos="<NULL>"):
    pos_index = {pos for pos_tags in all_pos for pos in pos_tags}
    pos_index = {pos: idx for idx, pos in enumerate(sorted(pos_index), start=1)}
    pos_index[null_pos] = 0

    return pos_index

word_index = words_to_idx(pd.concat([train_df, dev_df, test_df], sort=False)["words"], w2v)
pos_index = pos_to_idx(pd.concat([train_df, dev_df, test_df], sort=False)["pos"])

print(f"Vocab length: {len(word_index)} - PoS length: {len(pos_index)}")

Vocab length: 63757 - PoS length: 18
CPU times: user 14.3 s, sys: 469 ms, total: 14.8 s
Wall time: 7.58 s


In [24]:
%%time
def sequence_padding(df, word_index, pos_index, max_len):
    return (
        pad_sequences(
            df["words"].apply(
                lambda words: [word_index.get(word, word_index["<UNK>"]) for word in words]
            ).tolist(), maxlen=max_len
        ),
        pad_sequences(
            df["pos"].apply(
                lambda pos: [pos_index.get(p) for p in pos]
            ).tolist(), maxlen=max_len
        )
    )

train_word_sequences, train_pos_sequences = sequence_padding(
    train_df, word_index, pos_index, max_sequence_len
)

dev_word_sequences, dev_pos_sequences = sequence_padding(
    dev_df, word_index, pos_index, max_sequence_len
)

test_word_sequences, test_pos_sequences = sequence_padding(
    test_df, word_index, pos_index, max_sequence_len
)

CPU times: user 22.3 s, sys: 113 ms, total: 22.4 s
Wall time: 22.4 s


In [27]:
%%time

train_target = to_categorical(
    train_df["target"].tolist(),
    num_classes=lbl_enc.classes_.shape[0]
)

dev_target = to_categorical(
    dev_df["target"].tolist(),
    num_classes=lbl_enc.classes_.shape[0]
)

CPU times: user 342 ms, sys: 1.7 s, total: 2.04 s
Wall time: 2.04 s


In [40]:
%%time
def get_embedding_matrix(word_index, w2v):
    embedding_matrix = np.zeros((len(word_index), w2v.vector_size))

    for word, i in word_index.items():
        if word in w2v and word not in {"<NULL>", "<UNK>"}:
            embedding_matrix[i] = w2v[word]
        if word == "<UNK>":
            embedding_matrix[i] = np.random.normal(size=(w2v.vector_size,))

    return embedding_matrix

word_embedding_matrix = get_embedding_matrix(word_index, w2v)

CPU times: user 245 ms, sys: 44.3 ms, total: 290 ms
Wall time: 288 ms


# CNN Building

In [68]:
def build_cnn(word_vocab_size, word_vector_size, word_embedding_matrix, 
              pos_vocab_size, pos_vector_size, output_size, max_sequence_len,
              filters, filter_count, network_size, layer_size,
              activation="relu", padding="valid"):
    word_embedding_layer = Embedding(word_vocab_size, word_vector_size,
                                     weights=[word_embedding_matrix],
                                     input_length=max_sequence_len,
                                     trainable=False)
    pos_embedding_layer = Embedding(pos_vocab_size, pos_vector_size,
                                    embeddings_initializer="truncated_normal",
                                    input_length=max_sequence_len)

    word_sequence_input = Input(shape=(max_sequence_len,))
    word_embedded_sequences = word_embedding_layer(word_sequence_input)

    pos_sequence_input = Input(shape=(max_sequence_len,))
    pos_embedded_sequences = pos_embedding_layer(pos_sequence_input)
    
    embedded_sequences = Concatenate()([word_embedded_sequences, pos_embedded_sequences])

    layers = []
    for filter_size in filters:
        layer = Conv1D(
            filter_count,
            filter_size,
            activation=activation,
            padding=padding
        )(embedded_sequences)
        layer = GlobalMaxPooling1D()(layer)
        layers.append(layer)

    layer = Concatenate()(layers)    

    for _ in range(network_size):
        layer = Dense(layer_size, activation=activation)(layer)

    preds = Dense(output_size, activation="softmax")(layer)
    model = Model([word_sequence_input, pos_sequence_input], preds)

    return model

model = build_cnn(
    word_vocab_size=len(word_index),
    word_vector_size=w2v.vector_size, 
    word_embedding_matrix=word_embedding_matrix,
    pos_vocab_size=len(pos_index),
    pos_vector_size=pos_vector_size,
    output_size=lbl_enc.classes_.shape[0],
    max_sequence_len=max_sequence_len,
    filters=filters,
    filter_count=filter_count,
    network_size=network_size,
    layer_size=layer_size,
    activation=activation,
    padding=padding
)

model.compile(
    optimizer="nadam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 20, 300)      19127100    input_11[0][0]                   
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 20, 3)        54          input_12[0][0]                   
____________________________________________________________________________________________

# Fitting the CNN

In [ ]:
model.fit(
    x=(train_word_sequences, train_pos_sequences),
    y=train_target,
    batch_size=1024,
    epochs=10,
    validation_data=(
        (dev_word_sequences, dev_pos_sequences),
        dev_target
    ),
    validation_freq=5
)

Train on 918295 samples, validate on 499625 samples
Epoch 1/10
918295/918295 [==============================] - 89s 96us/sample - loss: 2.4571 - accuracy: 0.5635
Epoch 2/10
918295/918295 [==============================] - 81s 89us/sample - loss: 1.2266 - accuracy: 0.7556
Epoch 3/10
 91136/918295 [=>............................] - ETA: 1:14 - loss: 1.0641 - accuracy: 0.7847